In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seoul2021.csv')
df.head(1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2021,1,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,626851391.0,25502,...,157,15667,7253,108,1098,2013,4344,7792,7568,28


In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import matplotlib.pyplot as plt
import requests
import xml, xmltodict
import json
%matplotlib inline 

In [ ]:
#대분류(외식,서비스,소매)로 나누기
df.loc[df['서비스_업종_코드'].str.contains('CS1'),'분류'] = '외식업'
df.loc[df['서비스_업종_코드'].str.contains('CS2'),'분류'] = '서비스업'
df.loc[df['서비스_업종_코드'].str.contains('CS3'),'분류'] = '소매업'
df.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,분류
0,2021,1,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,626851391.0,25502,...,15667,7253,108,1098,2013,4344,7792,7568,28,외식업
1,2021,1,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,122957138.0,12039,...,6679,4293,52,926,1222,2698,3404,2670,9,외식업
2,2021,1,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,5093349.0,177,...,124,36,0,0,9,18,62,71,3,외식업
3,2021,1,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,13507733.0,1359,...,356,881,0,243,260,213,288,235,5,외식업
4,2021,1,A,골목상권,2110001,이북5도청사,CS200001,일반교습학원,37551380.0,128,...,56,72,0,0,8,96,8,16,3,서비스업


In [ ]:
#필요한 열만 가져오기

food = df[df['분류']=='외식업']

food = food[['상권_코드','상권_코드_명', '서비스_업종_코드_명', '분기당_매출_금액', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액','화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액','시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액','연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '점포수','분류']]
food['분류'].unique()

array(['외식업'], dtype=object)

In [ ]:
#분기별로 한 점포당 매출금액 구하기
div = ['분기당_매출_금액', '월요일_매출_금액','화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액','시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액','연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']

for i in div:
  food[i] = food[i] / food['점포수']

food.rename(columns={'분기당_매출_금액':'한_점포당_분기별_매출_금액'}, inplace=True)

food.head(2)

,상권_코드,상권_코드_명,서비스_업종_코드_명,한_점포당_분기별_매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,...,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,점포수,분류
0,2110001,이북5도청사,한식음식점,2.238755e+07,367617856.0,259233535.0,2.182818e+06,2.913635e+06,2.900728e+06,2.533008e+06,...,1.319211e+07,6.034029e+06,33387.785714,659016.392857,1.556685e+06,3.476915e+06,6.761393e+06,6.738743e+06,28,외식업
1,2110001,이북5도청사,분식전문점,1.366190e+07,61070094.0,61887044.0,9.568364e+05,1.696012e+06,1.181039e+06,1.718316e+06,...,7.112296e+06,5.297650e+06,27969.000000,517897.000000,1.123617e+06,2.790576e+06,5.361795e+06,2.588091e+06,9,외식업


In [ ]:
# Open API 데이터 가져오기
key = '546f7464586a756e3733666b687678'
url1 = 'http://openapi.seoul.go.kr:8088/{}/xml/TbgisTrdarRelm/1/1000/'.format(key)
url2 = 'http://openapi.seoul.go.kr:8088/{}/xml/TbgisTrdarRelm/1001/1671/'.format(key)

# 1 ~ 1000 행
response1 = requests.get(url1)
content1 = response1.content
dict1 = xmltodict.parse(content1)
jsonString1 = json.dumps(dict1['TbgisTrdarRelm']['row'], ensure_ascii= False)
jsonObj1 = json.loads(jsonString1)

# 1001 ~ 1672 행
response2 = requests.get(url2)
content2 = response2.content
dict2 = xmltodict.parse(content2)
jsonString2 = json.dumps(dict2['TbgisTrdarRelm']['row'], ensure_ascii= False)
jsonObj2 = json.loads(jsonString2)

# for item in jsonObj2:
#     print(item['TRDAR_SE_CD'], item['TRDAR_SE_CD_NM'], item['TRDAR_CD'],
#         item['TRDAR_CD_NM'],item['XCNTS_VALUE'], item['YDNTS_VALUE'],
#         item['SIGNGU_CD'], item['ADSTRD_CD'], item['STDR_YM_CD'])

list_1 = []
list_2 = []
list_3 = []

for item in jsonObj1:
    list_1.append(item['TRDAR_CD'])
    list_2.append(item['XCNTS_VALUE'])
    list_3.append(item['YDNTS_VALUE'])

for item in jsonObj2:
    list_1.append(item['TRDAR_CD'])
    list_2.append(item['XCNTS_VALUE'])
    list_3.append(item['YDNTS_VALUE'])

In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.8 MB 3.0 MB/s 


In [ ]:
from pyproj import Proj, transform

In [ ]:
# Projection 정의
proj_1 = Proj(init='epsg:5181')

# WGS84 경위도: GPS가 사용하는 좌표계 EPSG:4326
proj_2 = Proj(init='epsg:4326')

list_4 = []
list_5 = []

for item in list_2:
    list_4.append(int(item))

for item in list_3:
    list_5.append(int(item))

x_, y_ = transform(proj_1, proj_2, list_4, list_5)

dict_for_dataframe = {'상권_코드':list_1, 'X': x_, 'Y': y_}
coordinate = pd.DataFrame(dict_for_dataframe)
coordinate

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-11-41145ca1da7a>:16: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x_, y_ = transform(proj_1, proj_2, list_4, list_5)


,상권_코드,X,Y
0,2110753,126.908844,37.448687
1,2110427,127.070978,37.634469
2,2110755,126.912978,37.451006
3,2110739,126.901564,37.441482
4,2111017,127.115614,37.509060
...,...,...,...
1666,1001493,127.011025,37.567312
1667,1001494,126.997374,37.569998
1668,1001495,127.115275,37.516476
1669,1001496,127.060059,37.511003


In [ ]:
pro_food = food[['상권_코드','상권_코드_명']]
pro_food['상권_코드'] = pro_food['상권_코드'].astype('int64')
coordinate['상권_코드'] = coordinate['상권_코드'].astype('int64')
new_coord = pd.merge(pro_food, coordinate, on='상권_코드')
new_coord.drop(columns=['상권_코드'], inplace=True)
new_coord.drop_duplicates(['상권_코드_명'], inplace=True)
new_coord.head()

<ipython-input-18-b12f9fc550da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pro_food['상권_코드'] = pro_food['상권_코드'].astype('int64')


,상권_코드_명,X,Y
0,이북5도청사,126.956884,37.609345
16,독립문역 1번,126.957209,37.575540
33,세검정초등학교,126.958801,37.604489
53,대신고등학교,126.960550,37.573289
60,세검정,126.962120,37.600950


In [ ]:
# 전체 외식업 한점포당 1년 매출 TOP10
eatout = food.drop(columns=['상권_코드','점포수','서비스_업종_코드_명'])
eatout = eatout.groupby(['상권_코드_명']).sum()
eatout.rename(columns={'한_점포당_분기별_매출_금액':'외식업_연매출_금액'}, inplace=True)
final_eatfood = pd.merge(eatout, new_coord, on='상권_코드_명')
final_eatfood.to_csv('totalTOP10',index=False)

,상권_코드_명,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,...,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,X,Y
0,4.19민주묘지역 2번,4.122043e+08,5.074806e+09,3.190069e+09,4.365845e+07,5.135672e+07,5.273364e+07,4.568431e+07,5.864800e+07,7.285429e+07,...,2.266353e+08,1.722508e+08,2.597306e+06,5.515224e+07,6.652854e+07,8.507251e+07,1.069288e+08,8.260671e+07,127.012544,37.649834
1,63빌딩,3.907348e+08,8.919831e+09,3.216556e+09,5.752587e+07,5.696745e+07,5.921136e+07,5.563498e+07,5.486659e+07,5.646218e+07,...,1.674009e+08,1.227010e+08,1.426834e+06,4.143750e+07,1.076853e+08,7.316587e+07,3.788742e+07,2.849905e+07,126.939436,37.519778
2,DMC(디지털미디어시티),1.251771e+09,8.348779e+10,1.159614e+10,1.991759e+08,2.270928e+08,2.215268e+08,2.305169e+08,2.151906e+08,8.814911e+07,...,4.100654e+08,3.358330e+08,6.256755e+06,1.673500e+08,2.299935e+08,2.110270e+08,1.018148e+08,2.945637e+07,126.889949,37.579506
3,GS강동자이아파트,4.881138e+08,2.483070e+09,1.322897e+09,5.967703e+07,6.742865e+07,5.628591e+07,6.013578e+07,7.810087e+07,8.804023e+07,...,2.518722e+08,1.876911e+08,2.492833e+06,5.665300e+07,8.762309e+07,1.111311e+08,1.188163e+08,6.284693e+07,127.147464,37.537920
4,G타워,5.666744e+08,4.674009e+09,5.739245e+08,8.513741e+07,9.858214e+07,1.003136e+08,1.067211e+08,1.092430e+08,4.254189e+07,...,2.929081e+08,1.415984e+08,7.655191e+05,1.205939e+08,1.544198e+08,9.163787e+07,5.126273e+07,1.582672e+07,126.896787,37.480537


In [ ]:
# 전체 외식업 TOP10 상권 보기
total10 = final_eatfood.sort_values('외식업_연매출_금액',ascending=False).reset_index(drop=True).head(10)
total10

,상권_코드_명,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,...,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,X,Y
0,강남 마이스 관광특구,5.527805e+09,7.256726e+10,3.095753e+10,6.709933e+08,7.098114e+08,7.639630e+08,7.764800e+08,8.801755e+08,9.229621e+08,...,2.037686e+09,2.525007e+09,5.102284e+07,1.368569e+09,1.486457e+09,8.644149e+08,5.258099e+08,2.664197e+08,127.060059,37.511003
1,삼성역,3.793065e+09,1.291144e+11,3.461033e+10,5.119195e+08,5.570578e+08,5.789105e+08,5.959601e+08,6.285989e+08,5.046042e+08,...,1.445644e+09,1.545293e+09,2.250751e+07,8.492628e+08,1.017208e+09,5.921340e+08,3.472807e+08,1.625433e+08,127.062761,37.509281
2,고속터미널(고속터미널역),3.578846e+09,3.752608e+10,1.714538e+10,4.309896e+08,4.224788e+08,4.764693e+08,4.724266e+08,5.675759e+08,6.508790e+08,...,1.451944e+09,1.893413e+09,3.912406e+07,9.512657e+08,8.690255e+08,6.793963e+08,5.299125e+08,2.766330e+08,127.003981,37.505008
3,경기고교사거리(경기고교앞사거리),3.451394e+09,1.472292e+10,4.445941e+09,4.886747e+08,4.678612e+08,5.575893e+08,5.261019e+08,5.570839e+08,4.575849e+08,...,1.183392e+09,7.882343e+08,3.977846e+06,1.703989e+08,3.843008e+08,4.248915e+08,4.833660e+08,5.046915e+08,127.056572,37.520862
4,잠실역,2.915209e+09,8.785881e+10,3.042163e+10,3.862629e+08,4.029484e+08,4.243240e+08,4.664172e+08,4.558468e+08,4.398801e+08,...,1.244547e+09,1.303922e+09,3.233687e+07,5.887727e+08,7.217856e+08,5.861680e+08,3.945035e+08,2.249022e+08,127.100703,37.514273
5,상봉역,2.806588e+09,3.736097e+10,2.076177e+10,3.180455e+08,3.193232e+08,3.474246e+08,3.279805e+08,4.054796e+08,5.616445e+08,...,1.481443e+09,1.280140e+09,7.120837e+07,7.205775e+08,6.665713e+08,5.830626e+08,4.589930e+08,2.611839e+08,127.088074,37.596112
6,신도림역,2.771879e+09,5.557451e+10,2.003865e+10,3.706302e+08,3.962268e+08,4.112362e+08,4.189480e+08,4.661008e+08,4.254680e+08,...,1.185939e+09,1.472557e+09,2.698834e+07,7.683821e+08,7.597162e+08,5.036854e+08,3.782795e+08,2.214446e+08,126.890744,37.508066
7,엔터식스 강남점,2.534534e+09,4.932030e+10,2.296069e+10,3.042138e+08,3.195246e+08,3.373534e+08,3.506566e+08,4.047815e+08,4.374383e+08,...,1.041474e+09,1.232206e+09,2.183479e+07,6.598541e+08,6.177189e+08,4.045778e+08,3.622464e+08,2.074483e+08,127.007442,37.505197
8,서울시청,2.422068e+09,5.493517e+10,8.351337e+09,3.469739e+08,4.153029e+08,4.144917e+08,4.296605e+08,4.277447e+08,2.190273e+08,...,9.099932e+08,7.803200e+08,6.126186e+06,3.721046e+08,5.032155e+08,3.815043e+08,3.184480e+08,1.089146e+08,126.977667,37.567626
9,"평화시장(통일상가, 동화상가)",2.355345e+09,8.921245e+09,4.538885e+09,2.822314e+08,2.699862e+08,3.132518e+08,2.992256e+08,3.493426e+08,4.536287e+08,...,9.803014e+08,1.215725e+09,3.711203e+07,6.881391e+08,7.113115e+08,4.352437e+08,2.443171e+08,7.990341e+07,127.005863,37.568772


In [ ]:
d_eatout = food.drop(columns=['상권_코드','점포수'])
find_eatfood2 = d_eatout.groupby(['서비스_업종_코드_명','상권_코드_명']).sum()
find_eatfood2.rename(columns={'한_점포당_분기별_매출_금액':'외식업_연매출_금액'}, inplace=True)
find_eatfood2.head()

외식업_연매출_금액      주중_매출_금액      주말_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                   
분식전문점       4.19민주묘지역 2번   9.973888e+07  4.904646e+08  4.086928e+08   
            63빌딩           2.205884e+07  3.305726e+08  2.204197e+07   
            DMC(디지털미디어시티)  1.476247e+08  1.683536e+10  2.264233e+09   
            GS강동자이아파트      3.013934e+07  1.027481e+08  2.439720e+07   
            G타워            6.685492e+07  6.872253e+08  6.643019e+07   

                              월요일_매출_금액     화요일_매출_금액     수요일_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                   
분식전문점       4.19민주묘지역 2번   3.781067e+06  1.154937e+07  1.391045e+07   
            63빌딩           3.954925e+06  4.263924e+06  4.522038e+06   
            DMC(디지털미디어시티)  2.521406e+07  2.693039e+07  2.652829e+07   
            GS강동자이아파트      3.274378e+06  8.022260e+06  4.843268e+06   
            G타워            1.192588e+07  1.226589e+07  1.153976e+07   

                              목요일_매출_금액     금요일_매출_금액     토요일_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                   
분식전문점       4.19민주묘지역 2번   1.182972e+07  1.203071e+07  1.850196e+07   
            63빌딩           4.149245e+06  3.916534e+06  6.951772e+05   
            DMC(디지털미디어시티)  2.725019e+07  2.419427e+07  9.376522e+06   
            GS강동자이아파트      2.671562e+06  6.240311e+06  2.263256e+06   
            G타워            1.328022e+07  1.204738e+07  4.545159e+06   

                              일요일_매출_금액  ...  시간대_17~21_매출_금액  \
서비스_업종_코드_명 상권_코드_명                      ...                    
분식전문점       4.19민주묘지역 2번   2.813559e+07  ...     3.531556e+07   
            63빌딩           5.569947e+05  ...     2.448464e+06   
            DMC(디지털미디어시티)  8.130978e+06  ...     4.151464e+07   
            GS강동자이아파트      2.824300e+06  ...     4.776299e+06   
            G타워            1.250630e+06  ...     1.974745e+07   

                           시간대_21~24_매출_금액      남성_매출_금액      여성_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                      
분식전문점       4.19민주묘지역 2번      9.698799e+05  5.467190e+07  4.116352e+07   
            63빌딩              5.396111e+04  7.273648e+06  5.810492e+06   
            DMC(디지털미디어시티)     2.930006e+06  4.840184e+07  4.587346e+07   
            GS강동자이아파트         4.095038e+05  1.084414e+07  5.879464e+06   
            G타워               1.319014e+06  3.767339e+07  2.052843e+07   

                            연령대_10_매출_금액  연령대_20_매출_금액  연령대_30_매출_금액  \
서비스_업종_코드_명 상권_코드_명                                                    
분식전문점       4.19민주묘지역 2번   404825.928571  9.960276e+06  1.469936e+07   
            63빌딩            12447.188889  2.520695e+06  4.252659e+06   
            DMC(디지털미디어시티)  752928.987261  2.285137e+07  2.909001e+07   
            GS강동자이아파트      149157.666667  2.067758e+06  3.308152e+06   
            G타워             53553.055556  1.846320e+07  2.163630e+07   

                           연령대_40_매출_금액  연령대_50_매출_금액  연령대_60_이상_매출_금액  
서비스_업종_코드_명 상권_코드_명                                                     
분식전문점       4.19민주묘지역 2번   2.035607e+07  2.496565e+07     2.544924e+07  
            63빌딩           3.619898e+06  2.209496e+06     4.689443e+05  
            DMC(디지털미디어시티)  2.579891e+07  1.283828e+07     2.943800e+06  
            GS강동자이아파트      3.382663e+06  5.527028e+06     2.288840e+06  
            G타워            1.161823e+07  5.358675e+06     1.071865e+06  

[5 rows x 24 columns]

In [ ]:
#분기별데이터 합치기(세부 외식업 별 한점포당 1년 매출)
d_eatout = food.drop(columns=['상권_코드','점포수'])
detail_eatfood = d_eatout.groupby(['서비스_업종_코드_명','상권_코드_명']).sum().reset_index()
fdetail_eatfood = pd.merge(detail_eatfood, new_coord, on='상권_코드_명')
fdetail_eatfood.rename(columns={'한_점포당_분기별_매출_금액':'외식업_연매출_금액'}, inplace=True)
fdetail_eatfood.head()

,서비스_업종_코드_명,상권_코드_명,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,...,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,X,Y
0,분식전문점,4.19민주묘지역 2번,9.973888e+07,4.904646e+08,4.086928e+08,3.781067e+06,1.154937e+07,1.391045e+07,1.182972e+07,1.203071e+07,...,5.467190e+07,4.116352e+07,4.048259e+05,9.960276e+06,1.469936e+07,2.035607e+07,2.496565e+07,2.544924e+07,127.012544,37.649834
1,일식음식점,4.19민주묘지역 2번,1.864082e+07,4.151452e+07,1.440792e+07,3.266890e+06,3.220828e+06,3.110200e+06,2.605207e+06,1.635050e+06,...,8.754317e+06,9.315053e+06,3.224020e+05,5.607274e+06,3.074372e+06,2.273892e+06,3.182085e+06,3.609344e+06,127.012544,37.649834
2,치킨전문점,4.19민주묘지역 2번,2.595082e+07,1.415085e+08,7.988766e+07,3.048397e+06,3.350680e+06,2.471320e+06,3.481092e+06,4.328515e+06,...,1.496594e+07,9.781558e+06,1.209334e+05,3.182285e+06,4.466957e+06,6.889613e+06,6.306921e+06,3.780783e+06,127.012544,37.649834
3,커피-음료,4.19민주묘지역 2번,7.411106e+07,1.030813e+09,5.203655e+08,9.346153e+06,9.517197e+06,9.329955e+06,1.158418e+07,9.339553e+06,...,3.040680e+07,4.251002e+07,1.379158e+06,1.686984e+07,1.683465e+07,1.708091e+07,1.401697e+07,6.735289e+06,127.012544,37.649834
4,한식음식점,4.19민주묘지역 2번,1.059581e+08,2.359026e+09,1.597712e+09,1.236573e+07,1.286680e+07,1.349198e+07,8.898646e+06,1.552809e+07,...,6.334097e+07,3.747407e+07,1.956777e+05,6.685786e+06,1.252295e+07,1.944555e+07,3.185473e+07,3.011035e+07,127.012544,37.649834


In [ ]:
#세부 외식(10개업종)별 TOP10

store = ['한식음식점','중식음식점','일식음식점','양식음식점','제과점','패스트푸드점','치킨전문점','분식전문점','호프-간이주점','커피-음료']

detail_result = pd.DataFrame()

for i in store:
  df = fdetail_eatfood[fdetail_eatfood['서비스_업종_코드_명']==i].set_index('서비스_업종_코드_명')
  df = df.sort_values('외식업_연매출_금액', ascending=False).head(10)
  detail_result = detail_result.append(df)

detail_result

,상권_코드_명,외식업_연매출_금액,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,...,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,X,Y
서비스_업종_코드_명,,,,,,,,,,,,,,,,,,,,,
한식음식점,테크노상가(엘리시움),1.527457e+09,4.008876e+09,5.734940e+08,2.125973e+08,2.601018e+08,2.647990e+08,2.572741e+08,3.415199e+08,1.911647e+08,...,4.737722e+08,8.046154e+08,2.523363e+05,6.299774e+07,2.392931e+08,3.006494e+08,3.370962e+08,3.380987e+08,127.013142,37.568907
한식음식점,삼성중앙역 5번,1.189374e+09,1.621731e+10,3.523213e+09,1.612868e+08,1.814507e+08,2.094550e+08,2.110530e+08,2.144262e+08,1.216000e+08,...,3.443253e+08,1.434887e+08,2.146609e+05,3.594066e+07,1.383276e+08,1.726184e+08,9.923948e+07,4.147314e+07,127.055842,37.512356
한식음식점,제기동역 1번,6.700736e+08,2.723405e+09,1.460024e+09,8.720673e+07,8.415122e+07,8.800745e+07,8.823625e+07,8.857571e+07,1.139820e+08,...,3.422603e+08,2.650027e+08,5.039995e+05,2.321751e+07,5.707738e+07,1.007949e+08,1.470726e+08,2.785967e+08,127.033374,37.579787
한식음식점,잠원역,6.079325e+08,1.218359e+09,6.054381e+08,7.515913e+07,9.384941e+07,7.325202e+07,7.583947e+07,8.801973e+07,1.070244e+08,...,3.221749e+08,1.449984e+08,1.652415e+06,3.508212e+07,7.519174e+07,1.439228e+08,1.172943e+08,9.402987e+07,127.011390,37.512766
한식음식점,봉원사,5.985700e+08,3.172558e+09,2.410409e+08,1.600164e+08,6.514649e+07,9.954917e+07,7.879544e+07,1.550691e+08,3.015904e+07,...,3.467488e+07,5.973471e+07,8.662213e+05,1.992764e+07,1.524963e+07,1.599987e+07,2.656134e+07,1.580489e+07,126.946686,37.568508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
커피-음료,명동지하쇼핑센터(소공동지하쇼핑센터),3.575862e+08,8.676633e+08,2.050952e+08,5.580227e+07,5.578022e+07,5.690491e+07,6.013421e+07,6.059949e+07,3.515076e+07,...,2.207117e+08,1.151681e+08,7.298323e+05,8.681936e+07,1.594144e+08,6.153161e+07,2.392290e+07,3.461617e+06,126.981449,37.563689
커피-음료,도산공원북측,3.493070e+08,5.547772e+08,4.931439e+08,3.382255e+07,2.811167e+07,4.014706e+07,3.832685e+07,4.451760e+07,8.411897e+07,...,1.014157e+08,2.116905e+08,3.821132e+06,1.229422e+08,1.112354e+08,3.700836e+07,2.667533e+07,1.142373e+07,127.035092,37.525681
커피-음료,석계역 2번,3.418743e+08,2.637815e+09,1.283998e+09,4.430676e+07,4.089000e+07,4.574206e+07,4.485548e+07,5.423596e+07,6.164472e+07,...,1.699097e+08,1.712497e+08,8.063865e+06,1.234705e+08,8.251414e+07,7.553373e+07,4.174483e+07,9.832409e+06,127.063997,37.619129


In [ ]:
import folium
import math

In [ ]:
test_df = detail_result[0:10]

seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                       zoom_start=12)

for name, lat, lng, pop in zip(test_df.상권_코드_명,
                                test_df.Y,
                                test_df.X,
                                test_df.외식업_연매출_금액):
    folium.CircleMarker(
        [lat, lng],
        radius= math.sqrt(pop//1000000),
        color= 'black',
        fill_color='#ff000099',
        tooltip= '{} 상세정보'.format(name),
        popup=name).add_to(seoul_map)

display(seoul_map)